# Bisection for Mass Attenuation Coefficient and Thickness Optimization

### 1.1. Single Energy Bisection

In [ ]:
import Bisection_Library as BisLib

root_filename = "ROOT/root0.root"
root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

initial_energy = 10  # eV

thickness = 1e-6
thick_0 = thickness / 1e2  # mm 
thick_1 = thickness * 1e2  # mm

tolerance = 30

BisLib.SingleEnergyBisection(root_filename, root_structure, initial_energy, thick_0, thick_1, tolerance)

## 2. Multiple Energies Bisection

###  2.1. Energies from NIST CSV

In [ ]:
import Bisection_Library as BisLib

threads = 10

root_filename = "root0.root"
outputcsv_name = 'CadTel_map.csv'

root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

input_csv = 'CdTe_nist.csv'

tolerance = 8

BisLib.BisectionEnergiesNIST(threads, root_filename, outputcsv_name, root_structure, input_csv, tolerance)

### 2.2. Fixed Energy Step

In [ ]:
import Bisection_Library as BisLib

threads = 10

root_filename = "ROOT/root0.root"
outputcsv_name = 'aaaaaaaaaaaaaaaaaaaaaaa.csv'

root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

energies = [100000, 101000, 1000] # initial energy, final energy, energy step (in eV)

tolerance = 8

BisLib.BisectionFixedEnergyStep(threads, root_filename, outputcsv_name, root_structure, energies, tolerance)

## 3. Merge CSV's

In [ ]:
import os

def MergeCSV(directory, output_file):
    first_file = True
    # Sort the files in ascending order
    sorted_filenames = sorted(os.listdir(directory))

    # print(sorted_filenames)
    
    with open(output_file, 'w') as outfile:
        for filename in sorted_filenames:
            if filename.endswith('.csv') and filename != output_file:
                with open(os.path.join(directory, filename), 'r') as file:
                    if first_file:
                        outfile.write(file.read())  # Write the content including the header
                        first_file = False
                    else:
                        next(file)  # Skip the header
                        outfile.write(file.read())  # Write the rest of the content
                    outfile.write('\n')

directory = 'BUILD/ROOT'
output_file = 'Bone_map.csv'

MergeCSV(directory, output_file)

## 4. Plot Results

### 4.1. Plot Attenuation Coefficient vs NIST

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import Bisection_Library as BisLib

def Plot1_Coeff(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label):

    path_1 = os.path.join(directory, csv_1)
    path_2 = os.path.join(directory, csv_2)
    
    df_1 = pd.read_csv(path_1)
    df_2 = pd.read_csv(path_2)

    x1 = df_1[x_branch_1]
    y1 = df_1[y_branch_1]

    x2 = df_2[x_branch_2]
    y2 = df_2[y_branch_2]

    merged_df = pd.merge(df_1, df_2, on = 'Energy', suffixes = ('coeff_calc', 'coeff_real'))
    merged_df['Percentage_Error'] = abs((merged_df[y_branch_1] - merged_df[y_branch_2]) / merged_df[y_branch_2]) * 100
    global_percentage_error = merged_df['Percentage_Error'].mean()
    # print(f"Global Percentage Error: {global_percentage_error:.2f}%")

    merged_df = pd.merge(df_1, df_2, on='Energy', suffixes=('coeff_calc', 'coeff_real'))

    threshold_value = 1 #kev

    filtered_df = merged_df[merged_df['Energy'] > threshold_value].copy()
    filtered_df.loc[:, 'Percentage_Error'] = abs((filtered_df[y_branch_1] - filtered_df[y_branch_2]) / filtered_df[y_branch_2]) * 100
    percentage_error_filtered = filtered_df['Percentage_Error'].mean()
    # print(f"Percentage Error > {threshold_value} keV: {percentage_error_filtered:.2f}%")

    plt.figure(figsize=(10, 6))
    BisLib.PlotsFormatting()
    plt.grid(True, which='both', linestyle='--', linewidth=0.7)

    plt.plot(x1, y1, marker = 'o', markersize = 2, label = 'Calculated', color = 'blue')
    plt.plot(x2, y2, marker = 'x', markersize = 2, label = 'Real Data', color = 'red', alpha = 0.6)

    plt.xscale('log')
    plt.yscale('log')

    # plt.xlim(.01, 1)
    # plt.ylim(1000, 250000)

    plt.xlabel(x_label, labelpad = 7)
    plt.ylabel(y_label, labelpad = 8)
    plt.title(title, pad = 14)
    plt.legend()

    plt.figtext(0.14, 0.2, f'Global Percentage Error: {global_percentage_error:.2f}%', 
                fontsize = 12, bbox = dict(facecolor = 'white', alpha = 0.5))
    
    plt.figtext(0.14, 0.139, f'Percentage Error > {threshold_value} keV: {percentage_error_filtered:.2f}%', 
                fontsize = 12, bbox = dict(facecolor = 'white', alpha = 0.5))
    

    plt.savefig(title + '_coeff' + '.png', dpi = 400)
    plt.show()

In [ ]:
directory = 'BUILD/ROOT/'

csv_1 = 'CadTel_map.csv'
x_branch_1 = "Energy"
y_branch_1 = 'AtCoefficient'

csv_2 = 'CdTe_nist.csv'
x_branch_2 = "Energy"
y_branch_2 = 'AtCoeff'

title   = r"Wolfram ($W, Z=74$)"
x_label = r"Energy ($KeV$)"
y_label = r"Mass Attenuation Coefficient ($cm^2/g$)"

Plot1_Coeff(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label)

### 4.2. Plot Coefficient and Thickness vs. Energy

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import Bisection_Library as BisLib

def Plot_DoubleAxis(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label, save_as):

    path_1 = os.path.join(directory, csv_1)
    path_2 = os.path.join(directory, csv_2)
    
    df_1 = pd.read_csv(path_1)
    df_1.columns = df_1.columns.str.strip()

    df_2 = pd.read_csv(path_2)
    df_2.columns = df_2.columns.str.strip()

    x1 = df_1[x_branch_1]
    y1 = df_1[y_branch_1]

    x2 = df_2[x_branch_2]
    y2 = df_2['Muscle']

    x3 = df_1[x_branch_1]
    y3 = df_1['Muscle']

    # merged_df = pd.merge(df_1, df_2, on='Energy', suffixes=('coeff_calc', 'coeff_real'))
    # merged_df['Percentage_Error'] = abs((merged_df[y_branch_1] - merged_df[y_branch_2]) / merged_df[y_branch_2]) * 100
    # global_percentage_error = merged_df['Percentage_Error'].mean()

    # threshold_value = 1  # keV

    # filtered_df = merged_df[merged_df['Energy'] > threshold_value].copy()
    # filtered_df.loc[:, 'Percentage_Error'] = abs((filtered_df[y_branch_1] - filtered_df[y_branch_2]) / filtered_df[y_branch_2]) * 100
    # percentage_error_filtered = filtered_df['Percentage_Error'].mean()

    fig, ax1 = plt.subplots(figsize = (10, 6))
    BisLib.PlotsFormatting()
    ax1.grid(True, which = 'both', linestyle = '--', linewidth = 0.7)
    
    ax1.plot(x1, y1,        marker = 'o', markersize = 2, label = 'Calculated Mass Attenuation Coefficient', color = 'blue')
    ax1.plot(x3, y3 * 1e6,  marker = 'o', markersize = 1,                                   color = 'green', alpha = 0.01)
    ax1.plot(x2, y2,        marker = 'x', markersize = 2, label = 'Real Data NIST',              color = 'red', alpha = 0.6)

    ax2 = ax1.twinx()  
    
    ax2.plot(x2, y2,        marker = 'x', markersize = 2,                                                color = 'red',   alpha = 0.01)
    ax2.plot(x3, y3 * 1e6,  marker = 'o', markersize = 1, label = 'Thickness = ln(1/2)/M.A.Coefficient', color = 'green', alpha= 0.5)

    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax2.set_yscale('log') 

    ax1.set_xlabel(x_label,                labelpad = 7)
    ax1.set_ylabel(y_label, fontsize = 14, labelpad = 8)
    ax2.set_ylabel('nm',    fontsize = 16, labelpad = 8)  

    ax1.set_title(title, pad = 14)
    ax1.legend(loc = 'upper left')
    ax2.legend(loc = 'upper right')

    # fig.text(0.14, 0.2, f'Global Percentage Error: {global_percentage_error:.2f}%', 
    #          fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
    
    # fig.text(0.14, 0.139, f'Percentage Error > {threshold_value} keV: {percentage_error_filtered:.2f}%', 
    #          fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
    
    plt.savefig(save_as + '.png', dpi=400)
    plt.show()

    print('Plot saved as', save_as + '.png')

In [ ]:
directory = '/Users/miguelcomett/geant4-v11.2.2/Estancia_G4/RESULTS/RADIOGRAFÍAS/MAC_data/'

csv_1 = 'Cmtt_Tissues_Small.csv'
x_branch_1 = "Energy"
y_branch_1 = 'Muscle'

csv_2 = 'Cmtt_Melbourne_AtCoeff.csv'
x_branch_2 = "Energy"
y_branch_2 = 'Skin'

title = r"hola"
x_label = r"Energy ($KeV$)"
y_label = r"($cm^2/g$)"

save_as = '?'

Plot_DoubleAxis(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label, save_as)